In [2]:
def read_raw_csv(file_name):
    with open(file_name, 'r',encoding="latin1") as file:
        csvreader = csv.reader(file)
        count = 0

        # count number of rows
        for row in csvreader:
          count += 1
        return count-1, len(row)  # -1 is for discarding header row.

In [4]:
import csv 

number_of_ques, number_of_columns = read_raw_csv(r'C:\AppliedAI\SearchEngine_QA-20220522T094532Z-001\SearchEngine_QA\archive\Questions.csv')
print(number_of_ques, number_of_columns)
number_of_ans, number_of_columns = read_raw_csv(r'C:\AppliedAI\SearchEngine_QA-20220522T094532Z-001\SearchEngine_QA\archive\Answers.csv')
print(number_of_ans, number_of_columns)

1264216 7
1048575 6


In [7]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

#es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es = Elasticsearch("http://localhost:9200")
if es.ping():
  print('Connected to ES!')
else:
  print('Could not connect!')

Connected to ES!


In [18]:
import json
import time
import tensorflow as tf
import tensorflow_hub as hub

In [29]:
# Mapping: Structure of the index
# Property/Field: name and type  
b = {"mappings": {
        "properties": {
            "title": {
              "type": "text"
            },
            "title_vector": {
              "type": "dense_vector",
              "dims": 512
        }
    }
     }
   }
ret = es.indices.create(index='questions-index', ignore=400, body=b) #400 caused by IndexAlreadyExistsException

C:\Users\Nikita\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  from ipykernel import kernelapp as app
C:\Users\Nikita\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  from ipykernel import kernelapp as app


In [24]:
#load USE4 model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [30]:
#Insert 200000 datapoints into elastic database
NUM_QUESTIONS_INDEXED = 200000

# Col-Names: Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
cnt=0
with open(r'C:\AppliedAI\SearchEngine_QA-20220522T094532Z-001\SearchEngine_QA\archive\Questions.csv', encoding="latin1") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',' )
    next(readCSV, None)  # skip the headers 
    for row in readCSV:
        #print(row[0], row[5])
        doc_id = row[0];
        title = row[5];
        vec = tf.make_ndarray(tf.make_tensor_proto(embed([title]))).tolist()[0]

        b = {"title":title,
            "title_vector":vec,
            }
        
        res = es.index(index="questions-index", id=doc_id, body=b)
        #print(res)

        # keep count of # rows processed
        cnt += 1
        if cnt == NUM_QUESTIONS_INDEXED:
            break;

C:\Users\Nikita\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
